In [3]:
import sys
sys.path.append('../')

import numpy as np
import pandas as pd
import tempfile
import json
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="darkgrid")
%matplotlib inline

import logging
logging.getLogger().setLevel(logging.INFO)

from banditpylib import trials_to_dataframe
from banditpylib.arms import GaussianArm
from banditpylib.bandits import MultiArmedBandit
from banditpylib.protocols import SinglePlayerProtocol
from banditpylib.learners.mab_fcbai_learner import ExpGap, LilUCBHeuristic, TrackAndStop, BatchRacing, BatchTrackAndStop
from banditpylib.utils import argmax_or_min_tuple, argmax_or_min, argmax_or_min_tuple_second

In [4]:
# batch_sizes = [4, 8, 32]
# rhos= [0, 1e1, 1e3, 1e5]

# # {
# # "m1": [0.7, 0.4, 0.1],
# # "m2": [0.9, 0.7, 0.4, 0.1],
# # "m3": [0.9, 0.7, 0.5, 0.4, 0.45, 0.4, 0.3, 0.2]
# # }

# _means = ["m1", "m2", "m3"]#

In [ ]:
trials = 3
confidence = 0.99
means =  [0.9, 0.7, 0.5, 0.46, 0.45, 0.454, 0.453, 0.4]#, 0.33, 0.32, 0.3, 0.25, 0.24, 0.2] [0.9, 0.7, 0.5, 0.4, 0.1] #
max_pulls=50000
std=1
batch_size=32
gamma = 1e-4
gammas =  [1e-7, 1e-3, 1e-5] 
#num_switches = len(means)-1
num_switches = 2

switches = [4, 1,2,  0]
for num_switches in switches:
#for gamma in gammas:
    arms = [GaussianArm(mu=mean, std=std) for mean in means]
    bandit = MultiArmedBandit(arms=arms)
    learners = [
    #             ExpGap(arm_num=len(arms), confidence=confidence, threshold=3,  name='Exponential-Gap Elimination'),
    #             BatchRacing(arm_num=len(arms), confidence=confidence, max_pulls=max_pulls, 
    #                             k=1, b=batch_size, r=int(batch_size/2), name='BatchRacing'),

        LilUCBHeuristic(arm_num=len(arms), confidence=confidence, max_pulls=max_pulls, name='Heuristic lilUCB'),
        BatchTrackAndStop(arm_num=len(arms), num_switches=num_switches, confidence=confidence, batch_size=batch_size, gamma=gamma, 
                            max_pulls=max_pulls,  name='Batched Track and stop'),    
    #         TrackAndStop(arm_num=len(arms), confidence=confidence, tracking_rule="D", 
    #                             max_pulls=max_pulls,  name='Track and stop D-Tracking'),
    #         TrackAndStop(arm_num=len(arms), confidence=confidence, tracking_rule="C", 
    #                             max_pulls=max_pulls,  name='Track and stop C-Tracking')

               ]

    # For each setup, we run 20 trials
    trials=trials
    temp_file = tempfile.NamedTemporaryFile()

    game = SinglePlayerProtocol(bandit=bandit, learners=learners)

    # Add `debug=True` for debugging purpose
    game.play(trials=trials, output_filename=temp_file.name)

    # print("Sparsity: ", bts.sparsity)

    # Start playing the game
    trials_df = trials_to_dataframe(temp_file.name)
    trials_df.to_csv("csv_files/trial_df_gamma_" + str(gamma)+ "_num_switches_" + str(num_switches) + "_.csv", index=False)  # `index=False` ensures that the index is not saved in the CSV.

    # rho=1e3
# gammas= [0, 1e1, 1e3, 1e5]
# #for num_switches in switches:
# for gamma in gammas:
#     arms = [GaussianArm(mu=mean, std=std) for mean in means]
#     bandit = MultiArmedBandit(arms=arms)
#     learners = [
#     #             ExpGap(arm_num=len(arms), confidence=confidence, threshold=3,  name='Exponential-Gap Elimination'),
#     #             BatchRacing(arm_num=len(arms), confidence=confidence, max_pulls=max_pulls, 
#     #                             k=1, b=batch_size, r=int(batch_size/2), name='BatchRacing'),

#         LilUCBHeuristic(arm_num=len(arms), confidence=confidence, max_pulls=max_pulls, name='Heuristic lilUCB'),
#         BatchTrackAndStop(arm_num=len(arms), num_switches=num_switches, confidence=confidence, batch_size=batch_size, rho=rho, gamma=gamma, 
#                             max_pulls=max_pulls,  name='Batched Track and stop'),        
# #         TrackAndStop(arm_num=len(arms), confidence=confidence, tracking_rule="D", 
# #                             max_pulls=max_pulls,  name='Track and stop D-Tracking'),
# #         TrackAndStop(arm_num=len(arms), confidence=confidence, tracking_rule="C", 
# #                             max_pulls=max_pulls,  name='Track and stop C-Tracking')

#                ]

#     # For each setup, we run 20 trials
#     trials=trials
#     temp_file = tempfile.NamedTemporaryFile()

#     game = SinglePlayerProtocol(bandit=bandit, learners=learners)
#     # Start playing the game
#     # Add `debug=True` for debugging purpose
#     game.play(trials=trials, output_filename=temp_file.name)

#     trials_df = trials_to_dataframe(temp_file.name)
#     trials_df.to_csv("csv_files/trial_df_rho_" + str(rho) + "_gamma_" + str(gamma) + "_.csv", index=False)  # `index=False` ensures that the index is not saved in the CSV.

INFO:absl:start Heuristic lilUCB's play with multi_armed_bandit
INFO:absl:Heuristic lilUCB's play with multi_armed_bandit runs 2.35 seconds.
INFO:absl:start Batched Track and stop's play with multi_armed_bandit


In [ ]:
# game = SinglePlayerProtocol(bandit=bandit, learners=learners)
# # Start playing the game
# # Add `debug=True` for debugging purpose
# game.play(trials=trials, output_filename=temp_file.name)

In [ ]:
# trials_df = trials_to_dataframe(temp_file.name)
# trials_df.to_csv("trial_df_rho_" + str(rho) + "_batch_size_" + str(batch_size) + "_.csv", index=False)  # `index=False` ensures that the index is not saved in the CSV.

In [ ]:
# trials_df.tail(100)

In [ ]:
trials_df['confidence'] = 0.99

In [ ]:
fig = plt.figure()
ax = plt.subplot(111)
sns.barplot(x='confidence', y='total_actions', hue='learner', data=trials_df)
plt.ylabel('pulls')
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

In [ ]:
fig = plt.figure()
ax = plt.subplot(111)
sns.barplot(x='confidence', y='rounds', hue='learner', data=trials_df)
plt.ylabel('rounds')
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

In [ ]:
arm_num=8
w0 = np.ones(arm_num) / arm_num
r0 = 1
initial_guess = np.append(w0, r0)

w_values = np.zeros((arm_num, w0.shape[0]))
fx_values = np.zeros((arm_num))

In [ ]:
w_values[0]